In [1]:
import requests,time
import xml.dom.minidom as m
import xml.etree.ElementTree as ET
import pandas as pd
requests.adapters.DEFAULT_RETRIES = 5
def post_infoes(pmid):
    if len(pmid)==0:
        return pd.DataFrame()
    pmidStr=','.join(pmid)
    print(len(pmid))
    r = requests.post(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    f"efetch.fcgi?db=pubmed&retmode=xml&id={pmidStr}"
    ,timeout=30
    )
    tree = ET.fromstring(r.text)
    res=[]

    for node in tree:
        for iter in node.iter("PMID"):
            id=ET.tostring(iter, method="text").decode()
            break
        for iter in node.iter("ArticleTitle"):
            title=ET.tostring(iter, method="text").decode()
            break
        abstract=[]
        for iter in node.iter("AbstractText"):     #exclude copyright
            s=ET.tostring(iter, method="text").decode()
            abstract.append(s)
        abstract=' '.join(abstract)
        f=''
        if not id:
            f+=' empty id'
        if not title:
            f+=' empty title'
        if not abstract:
            f+=' empty abstract'
        if f:
            print(str(id)+f)

        res.append({'id':id,'ArticleTitle':title,'AbstractText':abstract})
    return pd.DataFrame(res).set_index('id')

def post_ids(target,lenth=1000,year=2022,articleOnly=True):
    a='+article' if articleOnly else ''
    r = requests.post(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    f"esearch.fcgi?db=pubmed&term={target}+AND+{year}[pdat]{a}&retmode=xml&retmax={lenth}&retstart=0"
    )
    doc = m.parseString(r.text)
    ids = doc.getElementsByTagName("Id")
    idResult=[id.childNodes[0].wholeText for id in ids]
    return idResult

In [3]:
lenth=1000
batch=350
query='Alzheimer'
ids_az=post_ids(query,lenth,2022,True)
result=[]
for i in range(int(lenth/batch+1)):
    time.sleep(1)
    result.append(post_infoes(ids_az[i*batch:(i+1)*batch]))
result=pd.concat(result,axis=0)
print(result)
queryList=[query]*result.shape[0]
result['query']=queryList
result.to_json(f'./data/{query}.json',orient="index")

350
36316783 empty abstract
36310167 empty abstract
36284252 empty abstract
36284251 empty abstract
36281688 empty abstract
36281687 empty abstract
36281660 empty abstract
36281659 empty abstract
36271398 empty abstract
36268432 empty abstract
36266381 empty abstract
36260879 empty abstract
36239480 empty abstract
36238082 empty abstract
350
36224042 empty abstract
36218862 empty abstract
36214803 empty abstract
36195891 empty abstract
36192348 empty abstract
36190718 empty abstract
36190701 empty abstract
36184585 empty abstract
36181001 empty abstract
36180246 empty abstract
36179926 empty abstract
36151476 empty abstract
300
36148860 empty abstract
36115352 empty abstract
36114241 empty abstract
36114240 empty abstract
36103896 empty abstract
36103266 empty abstract
36103250 empty abstract
36102768 empty abstract
36094832 empty abstract
36094251 empty abstract
36090866 empty abstract
                                               ArticleTitle  \
id                                   

In [4]:
query = 'cancer'
ids_cn = post_ids(query, lenth, 2022, True)
result = []
for i in range(int(lenth / batch + 1)):
    time.sleep(1)
    result.append(post_infoes(ids_cn[i * batch:(i + 1) * batch]))
result = pd.concat(result, axis=0)
print(result)
queryList = [query] * result.shape[0]
result['query'] = queryList
result.to_json(f'./data/{query}.json', orient="index")

350
36323504 empty abstract
36322873 empty abstract
36322839 empty abstract
36322273 empty abstract
36322272 empty abstract
36322095 empty abstract
36321985 empty abstract
36321983 empty abstract
36321979 empty abstract
36321785 empty abstract
36321685 empty abstract
36321521 empty abstract
36321400 empty abstract
36321342 empty abstract
36321306 empty abstract
36321301 empty abstract
36321298 empty abstract
36321297 empty abstract
36321294 empty abstract
36321287 empty abstract
36321264 empty abstract
36320882 empty abstract
36320099 empty abstract
350
36319872 empty abstract
36319870 empty abstract
36319869 empty abstract
36319868 empty abstract
36319855 empty abstract
36319698 empty abstract
36319551 empty abstract
36319517 empty abstract
36319448 empty abstract
36319447 empty abstract
36319446 empty abstract
36318738 empty abstract
36318150 empty abstract
36318058 empty abstract
36318057 empty abstract
36317775 empty abstract
36317575 empty abstract
36317444 empty abstract
36317390

In [9]:
same=set(ids_az)&set(ids_cn)
print('find_overlap:',same)

find_overlap: {'36321615'}
